### Gallery

In [ ]:
from holoext.xbokeh import Mod
import numpy as np
import pandas as pd
import holoviews as hv
import warnings

warnings.filterwarnings('ignore')  # bokeh deprecation warnings
hv.extension('bokeh')

In [ ]:
DATA_URL_FMT = (
    'http://mesonet.agron.iastate.edu/'
    'cgi-bin/request/daily.py?'
    'network=IL_ASOS&stations={0}&'
    'year1=2014&month1=1&day1=1&year2=2018&month2=1&day2=1'
)
STATIONS = ['CMI', 'DEC', 'MDW', 'ORD']

#### Download and process data from Illinois' stations (Champaign, Decatur, Midway, O'Hare).

In [ ]:
df_list = []
for station in STATIONS:
    data_url = DATA_URL_FMT.format(station)
    df = pd.read_csv(data_url, index_col='day', parse_dates=True)
    df.iloc[:, 1:] = df[df.columns[1:]].apply(
        pd.to_numeric, errors='coerce').fillna(0)
    df.index.name = 'date'
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['precip_cumsum_in'] = df['precip_in'].cumsum()
    df = df.reset_index()  # 7-day rolling average
    df_list.append(df)
df = pd.concat(df_list)
ds = hv.Dataset(df)
df.columns

### Overlay of Curves

In [ ]:
curve_list = []
for station in STATIONS:
    curve = hv.Curve(ds.select(station=station), ['date'], 'precip_cumsum_in', label=station)
    curve = curve.opts(style=dict(line_alpha=0.75))  # style the curves
    curve_list.append(curve)
curves_overlay = hv.Overlay(curve_list)

save_fp = 'html_output/gallery_station_precip'
Mod(
    xlabel='Date',
    ylabel='Cumulative Precip [in]',
    tools=['ypan', 'ywheel_zoom', 'save', 'reset']
).apply(curves_overlay, save=save_fp)

### Layout of HeatMaps

In [ ]:
# Automatic activation of colorbar and ranges
max_hmap = hv.HeatMap(
    ds.select(station='CMI'),
    ['year', 'month'],
    'max_temp_f',
    label='CMI Max Temp [F]'
)

min_hmap = hv.HeatMap(
    ds.select(station='CMI'),
    ['year', 'month'],
    'min_temp_f',
    label='CMI Min Temp [F]'
)

hmaps = max_hmap + min_hmap

zlim_dict = {
    'min_temp_f': (0, 80),
    'max_temp_f': (20, 100)
}

save_fp = 'html_output/gallery_cmi_temp'
Mod(
    xlabel='Year',
    ylabel='Month',
    colorbar_n=8,
    zlim=zlim_dict,
    tools=['save', 'hover'],
).apply(hmaps, save=save_fp)

### HoloMap of Points

In [ ]:
points = ds.to.points(['date', 'max_temp_f'], ['precip_in'], ['station']).opts(style={'Points': dict(alpha=0.35)})

save_fp = 'html_output/gallery_station_temperature_and_precipitation'
Mod(
    tools=['hover', 'save', 'ypan', 'ywheel_zoom'],
    title='Station Temperature and Precipitation',
    ylabel='Max Temp [F]',
    xlabel='Date',
    num_xticks=15,
    xrotation=35,
    size_index=2,
    color_index=2,
    scaling_factor=25,
    colorbar_n=5,
    colorbar_lim=(0, 5),
    colorbar_title='[in]'
).apply(points, save=save_fp)

### AdjointLayout of Scatter

In [ ]:
scatter = ds.select(station='ORD').to.scatter(
    'avg_wind_speed_kts',
    'avg_wind_drct',
    [], label='Station: ORD'
)

scatter_hists = scatter.hist(num_bins=5, dimension=['avg_wind_speed_kts','avg_wind_drct']).opts(
    plot={'Scatter': dict(width=725, height=500)},  # TODO: find a way to automate this
    style={'Scatter': dict(alpha=0.35, size=5),
           'Histogram': dict(alpha=0.75, line_color=None)}
).redim.label(
    avg_wind_speed_kts='Wind Speed [kts]',
    avg_wind_drct= 'Wind Dir. [deg]',
    avg_wind_speed_kts_frequency='Freq.',
    avg_wind_drct_frequency='Freq.'
)

save_fp = 'html_output/gallery_ord_wind_speed_wind_dir'
Mod().apply(scatter_hists, save=save_fp)

### GridMatrix of Bars

In [ ]:
def make_box(month, year):
    sub_ds = ds.select(month=month, year=year)
    text = hv.Text('CMI', 45, '{0:2d}/{1:s}'.format(
        month, str(year)[2:])).opts(style=dict(text_alpha=0.5, text_font_size='12px'))
    box = hv.BoxWhisker(sub_ds, 'station', 'max_rh')
    return box * text

boxes = {
    (month, year): make_box(month, year)
    for month in df['month'].unique()[2::4]  # to speed it up
    for year in df['year'].unique()
}

gridmatrix = hv.GridMatrix(boxes)
title = 'Max Relative Humidity Spread'
save_fp = 'html_output/gallery_station_temperature_and_precipitation'
Mod(
    axiswise=False,
    xlabel='Station',
    ylabel='Max RH [%]',
    autosize=True,
    title_format=title,
    label_scaler=0.6,
    merge_tools=True,
    tools=['save'],
    logo=False,
    plot_size=150  # width/height doesn't work in grid matrix
).apply(gridmatrix, save_fp)